In [1]:
!pip install --upgrade google-cloud-speech

     |████████████████████████████████| 112kB 821kB/s eta 0:00:01
     |████████████████████████████████| 102kB 1.4MB/s ta 0:00:01
     |████████████████████████████████| 512kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 143kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 887kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/03/74/3956721ea1eb4bcf7502a311fdaa60b85bd751de4e57d1943afe9b334141/googleapis_common_protos-1.52.0-py2.py3-none-any.whl
     |████████████████████████████████| 3.8MB 845kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/cb/3f/2f35751408c0b2a425aa48dd79301a48418858d3366f84c75d1aa42b6e2d/PyYAML-5.4.1-cp38-cp38-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/95/de/214830a981892a3e286c3794f41ae67a4495df1108c3da8a9f62159b9a9d/pyasn1_modules-0.2.8-py2.py3-none-any.whl
  Using cached https://files.pyth

In [64]:
!pip install --upgrade google-cloud-storage

     |████████████████████████████████| 102kB 1.1MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/22/52/4b086e0d15245b648e2b6d408fb2f2974a3a5a405de5d7fae25cd085a3fa/google_resumable_media-1.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ad/fc/6e8c449185cb8862af353c1164100ff75e32d55ba1de3baf9eaa01b7d2a9/google_cloud_core-1.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d8/4b5b7c93b38cf7bed500830bb4b3ba116f6f9b966c14c636f12c56f6cd37/google_crc32c-1.1.2-cp38-cp38-macosx_10_14_x86_64.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [102]:
PROJECT_ID = 'YOUR-PROJECT-ID'  # <--- CHANGE THIS

var = "https://console.developers.google.com/apis/api/speech.googleapis.com/overview?project="+PROJECT_ID
print("If this is the first time you are using the API in the project please enable the API via this link:\n"+ var)

If this is the first time you are using the API in the project please enable the API via this link:
https://console.developers.google.com/apis/api/speech.googleapis.com/overview?project=automl-demo-198411


# Make sure to enable the API
<img src="enable-api.png" width="800">

In [103]:
!export GOOGLE_APPLICATION_CREDENTIALS="/path/to/key.json"  # <--- CHANGE THIS

In [104]:
#check that the variable have been set correctly
GOOGLE_APPLICATION_CREDENTIALS

'AutoMLdemo-64ae368c71e2.json'

In [105]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/path/to/key.json" # <--- CHANGE THIS
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

AutoMLdemo-64ae368c71e2.json


In [106]:
from google.cloud import storage

BUCKET_NAME = "test-bucket-speech-n" # <--- CHANGE THIS

def create_bucket(bucket_name): 
    storage_client = storage.Client()
    bucket = storage_client.create_bucket(bucket_name)

    print("Bucket {} created.".format(bucket.name))
create_bucket(BUCKET_NAME)

Conflict: 409 POST https://storage.googleapis.com/storage/v1/b?project=automl-demo-198411&prettyPrint=false: You already own this bucket. Please select another name.

In [107]:
SOURCE_FILE_NAME = "Armstrong_Small_Step.ogg.mp3"
DEST_FILE_NAME = "Armstrong_Small_Step2.ogg.mp3"
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
    
upload_blob(BUCKET_NAME, SOURCE_FILE_NAME, DEST_FILE_NAME)

File Armstrong_Small_Step.ogg.mp3 uploaded to Armstrong_Small_Step2.ogg.mp3.


In [108]:
!gsutil ls gs://$BUCKET_NAME

gs://test-bucket-speech-n/2021020713284401905bff2636dcbb7e-partner-F3c69Fj6-972527143684.mp3
gs://test-bucket-speech-n/20210207132906019075a793a86e9da4-partner-htCGaDFf-972506009909.mp3
gs://test-bucket-speech-n/Armstrong_Small_Step.ogg.mp3
gs://test-bucket-speech-n/Armstrong_Small_Step2.ogg.mp3


In [109]:
from google.cloud import speech_v1p1beta1 as speech

GCS_URI = 'gs://' + BUCKET_NAME + '/' + DEST_FILE_NAME
print(GCS_URI)

def transcribe_sync(storage_uri):
    """
    Performs synchronous speech recognition on an audio file

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech.SpeechClient()


    # The language of the supplied audio
    language_code = "en-US"

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = speech.RecognitionConfig.AudioEncoding.MP3
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
        "model": "video"
    }
    audio = {"uri": storage_uri}

    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
        
transcribe_sync(GCS_URI)

gs://test-bucket-speech-n/Armstrong_Small_Step2.ogg.mp3
Transcript: step off the Lem now
Transcript:  that's one small step for man
Transcript:  one giant leap for mankind


In [112]:
from google.cloud import speech_v1p1beta1 as speech

GCS_URI = 'gs://' + BUCKET_NAME + '/' + DEST_FILE_NAME
print(GCS_URI)

def transcribe_async(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""

    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    encoding = speech.RecognitionConfig.AudioEncoding.MP3
    language_code = "en-US" #iw-IL
    sample_rate_hertz = 44100
    
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
        "use_enhanced":True,
        "model": "video"
    }

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=90)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u"Transcript: {}".format(result.alternatives[0].transcript))
        #print("Confidence: {}".format(result.alternatives[0].confidence))
transcribe_async(GCS_URI)        


gs://test-bucket-speech-n/Armstrong_Small_Step2.ogg.mp3
Waiting for operation to complete...
Transcript: step off the Lem now
Transcript:  that's one small step for man
Transcript:  one giant leap for mankind
